## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [1]:
!pip install sqlalchemy==1.3.9


     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.0 MB ? eta -:--:--
     - -------------------------------------- 0.3/6.0 MB ? eta -:--:--
     --- ------------------------------------ 0.5/6.0 MB 1.2 MB/s eta 0:00:05
     ----- ---------------------------------- 0.8/6.0 MB 1.4 MB/s eta 0:00:04
     ------- -------------------------------- 1.0/6.0 MB 1.2 MB/s eta 0:00:04
     -------- ------------------------------- 1.3/6.0 MB 1.2 MB/s eta 0:00:04
     ------------ --------------------------- 1.8/6.0 MB 1.3 MB/s eta 0:00:04
     -------------- ------------------------- 2.1/6.0 MB 1.4 MB/s eta 0:00:03
     --------------- ------------------------ 2.4/6.0 MB 1.4 MB/s eta 0:00:03
     ------------------- -------------------- 2.9/6.0 MB 1.4 MB/s eta 0:00:03
     ----------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
!pip install ipython-sql
!pip install ipython-sql prettytable

  Using cached sqlalchemy-2.0.46-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (9.5 kB)
Using cached sqlalchemy-2.0.46-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (3.3 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [3]:
%load_ext sql

In [4]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
!pip install -q pandas

In [6]:
%sql sqlite:///my_data1.db

In [7]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [8]:
df.columns

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='str')

In [9]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [10]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]







##### Display the names of the unique launch sites  in the space mission


In [11]:
%sql SELECT DISTINCT Launch_Site FROM SPACEXTABLE;


 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40





#####  Display 5 records where launch sites begin with the string 'CCA' 


In [12]:
%%sql
SELECT *
FROM spacextable
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5;


 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt







##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [21]:
%%sql
select 
    Customer,
    sum(PAYLOAD_MASS__KG_) AS total_payload_mass 
FROM SPACEXTABLE 
WHERE Customer='NASA (CRS)'
GROUP BY Customer 

 * sqlite:///my_data1.db
Done.


Customer,total_payload_mass
NASA (CRS),45596







##### Display average payload mass carried by booster version F9 v1.1


In [22]:
%%sql
SELECT 
   	Booster_Version,
    AVG(PAYLOAD_MASS__KG_) AS average_payload_mass
FROM SPACEXTABLE
WHERE Booster_Version='F9 v1.1'
GROUP BY Booster_Version;

 * sqlite:///my_data1.db
Done.


Booster_Version,average_payload_mass
F9 v1.1,2928.4




##### List the date when the first succesful landing outcome in ground pad was acheived.


 


In [24]:
df.tail()

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
96,2020-11-05,23:24:23,F9 B5B1062.1,CCAFS SLC-40,"GPS III-04 , Crew-1",4311,MEO,USSF,Success,Success
97,2020-11-16,0:27:00,F9 B5B1061.1,KSC LC-39A,"Crew-1, Sentinel-6 Michael Freilich",12500,LEO (ISS),NASA (CCP),Success,Success
98,2020-11-21,17:17:08,F9 B5B1063.1,VAFB SLC-4E,"Sentinel-6 Michael Freilich, Starlink 15 v1.0",1192,LEO,NASA / NOAA / ESA / EUMETSAT,Success,Success
99,2020-11-25,2:13:00,F9 B5 B1049.7,CCAFS SLC-40,"Starlink 15 v1.0, SpaceX CRS-21",15600,LEO,SpaceX,Success,Success
100,2020-12-06,16:17:08,F9 B5 B1058.4,KSC LC-39A,SpaceX CRS-21,2972,LEO (ISS),NASA (CRS),Success,Success




##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [38]:
%%sql
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (drone ship)'
  AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2







##### List the total number of successful and failure mission outcomes


In [36]:
%%sql
SELECT 
    Mission_Outcome,
    COUNT(*) AS Total_Count
FROM SPACEXTABLE
GROUP BY Mission_Outcome;


 * sqlite:///my_data1.db
Done.


Mission_Outcome,Total_Count
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [35]:
%%sql
SELECT
DISTINCT Landing_Outcome
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Landing_Outcome
Failure (parachute)
No attempt
Uncontrolled (ocean)
Controlled (ocean)
Failure (drone ship)
Precluded (drone ship)
Success (ground pad)
Success (drone ship)
Success
Failure






##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.


In [42]:
%%sql
SELECT DISTINCT "Booster_Version"
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (
    SELECT MAX(PAYLOAD_MASS__KG_)
    FROM SPACEXTABLE
);


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6





##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.




In [45]:
%%sql
SELECT
    substr(Date, 6, 2) AS Month,
    Landing_Outcome,
    Booster_Version,
    Launch_Site
FROM SPACEXTABLE
WHERE substr(Date, 0, 5) = '2015'
  AND Landing_Outcome = 'Failure (drone ship)';


 * sqlite:///my_data1.db
Done.


Month,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40







##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [46]:
%%sql
SELECT
    Landing_Outcome,
    COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
